# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff
using PseudoPotentialData

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice = T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.semicore.gth")
    atoms     = [ElementPsp(:He, pseudopotentials)]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions;
                      functionals=[:lda_x, :lda_c_vwn],
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770853852658                   -0.52    9.0    160ms
  2   -2.772147316806       -2.89       -1.32    1.0    117ms
  3   -2.772170256050       -4.64       -2.43    1.0    113ms
  4   -2.772170649160       -6.41       -3.13    1.0    104ms
  5   -2.772170722929       -7.13       -4.63    2.0    118ms
  6   -2.772170722993      -10.19       -4.70    2.0    120ms
  7   -2.772170723013      -10.69       -5.63    1.0    111ms
  8   -2.772170723015      -11.77       -6.71    2.0    125ms
  9   -2.772170723015      -14.12       -7.24    1.0    112ms
 10   -2.772170723015   +  -13.97       -7.81    1.0    113ms
 11   -2.772170723015      -14.31       -8.17    1.0    115ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770618463072                   -0.53    9.0    161ms
  2   -2.7

1.7735580054133593

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem. This leads to a density-functional perturbation
theory problem, which is automatically set up and solved in the background.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770806434875                   -0.52    9.0    677ms
  2   -2.772061665000       -2.90       -1.32    1.0   97.1ms
  3   -2.772083029406       -4.67       -2.44    1.0   97.3ms
  4   -2.772083344498       -6.50       -3.14    1.0    101ms
  5   -2.772083417690       -7.14       -4.39    2.0    118ms
  6   -2.772083417761      -10.15       -4.66    1.0    104ms
  7   -2.772083417808      -10.32       -5.64    1.0    108ms
  8   -2.772083417811      -11.69       -6.27    2.0    123ms
  9   -2.772083417811      -13.38       -6.56    1.0    114ms
 10   -2.772083417811      -13.86       -7.45    1.0    112ms
 11   -2.772083417811   +  -14.12       -8.02    2.0    129ms
Solving response problem
[ Info: GMRES linsolve starts with norm of residual = 4.19e+00
[ Info: GMRES linsolve in iteration 1; step 1: normres = 2.49e-01
[ Info: GMRES linsolve in iteration 1; st